In [1]:
%reset -f

## PFLOTRAN
import jupypft.model as mo
import jupypft.parameter as pm
import jupypft.attachmentRateCFT as arCFT
import jupypft.plotBTC as plotBTC

# Build the Case Directory

In [2]:
## Temperatures
Ref,Atm,Tin = pm.Real(tag="<initialTemp>",value=10.,units="C",mathRep="$$T_{0}$$"),\
              pm.Real(tag="<atmosphereTemp>",value=10,units="C",mathRep="$$T_{atm}$$"),\
              pm.Real(tag="<leakageTemp>",value=10., units="m³/d",mathRep="$$T_{in}$$")

In [3]:
LongDisp  = pm.Real(tag="<longDisp>",value=0.0,units="m",mathRep="$$\\alpha_L$$")

In [4]:
#Gradients
GX,GY,GZ  = pm.Real(tag="<GradientX>",value=0.,units="-",mathRep="$$\partial_x h$$"),\
            pm.Real(tag="<GradientY>",value=0.,units="-",mathRep="$$\partial_y h$$"),\
            pm.Real(tag="<Gradient>>",value=0.,units="-",mathRep="$$\partial_z h$$")

In [5]:
## Dimensions
LX,LY,LZ = pm.Real("<LenX>",value=200,units="m",mathRep="$$LX$$"),\
           pm.Real("<LenY>",value=100,units="m",mathRep="$$LY$$"),\
           pm.Real("<LenZ>",value=20,units="m",mathRep="$$LZ$$")

In [6]:
## Permeability
kX,kY,kZ = pm.Real(tag="<PermX>",value=1.0E-8,units="m²",mathRep="$$k_{xx}$$"),\
           pm.Real(tag="<PermY>",value=1.0E-8,units="m²",mathRep="$$k_{yy}$$"),\
           pm.Real(tag="<PermZ>",value=1.0E-8,units="m²",mathRep="$$k_{zz}$$")

theta = pm.Real(tag="<porosity>",value=0.35,units="adim",mathRep="$$\\theta$$")

In [7]:
## Extraction well
outX1,outX2 = pm.Real(tag="<outX1>",value=LX.value/2.,units="m",mathRep="$$x_{1,Q_{out}}$$"),\
              pm.Real(tag="<outX2>",value=LX.value/2.,units="m",mathRep="$$x_{2,Q_{out}}$$")

outY1,outY2 = pm.Real(tag="<outY1>",value=LY.value/2.,units="m",mathRep="$$y_{1,Q_{out}}$$"),\
              pm.Real(tag="<outY2>",value=LY.value/2.,units="m",mathRep="$$y_{2,Q_{out}}$$")

outZ1,outZ2 = pm.Real(tag="<outZ1>",value=LZ.value/2.   ,units="m",mathRep="$$z_{1,Q_{out}}$$"),\
              pm.Real(tag="<outZ2>",value=LZ.value - 1.0,units="m",mathRep="$$z_{2,Q_{out}}$$")

## Extraction rate
Qout = pm.Real(tag="<outRate>",value=-21.0,units="m³/d",mathRep="$$Q_{out}$$")

In [8]:
setbackDist = 40.

## Injection point
inX1,inX2 = pm.Real(tag="<inX1>",value=outX1.value + setbackDist,units="m",mathRep="$$x_{1,Q_{in}}$$"),\
            pm.Real(tag="<inX2>",value=outX2.value + setbackDist,units="m",mathRep="$$x_{2,Q_{in}}$$")

inY1,inY2 = pm.Real(tag="<inY1>",value=outY1.value + 0.0,units="m",mathRep="$$y_{1,Q_{in}}$$"),\
            pm.Real(tag="<inY2>",value=outY2.value + 0.0,units="m",mathRep="$$y_{2,Q_{in}}$$")

inZ1,inZ2 = pm.Real(tag="<inZ1>",value=LZ.value - 5.0,units="m",mathRep="$$z_{1,Q_{in}}$$"),\
            pm.Real(tag="<inZ2>",value=LZ.value - 1.0,units="m",mathRep="$$z_{2,Q_{in}}$$")

In [9]:
## Concentration
C0 = pm.Real("<initialConcentration>", value=1.0, units="mol/L")

## Injection rate
Qin = pm.Real(tag="<inRate>",value=0.24, units="m³/d",mathRep="$$Q_{in}$$")

In [10]:
## Grid
nX,nY,nZ = pm.Integer("<nX>",value=41,units="-",mathRep="$$nX$$"),\
           pm.Integer("<nY>",value=41 ,units="-",mathRep="$$nY$$"),\
           pm.Integer("<nZ>",value=1,units="-",mathRep="$$nZ$$")

dX,dY,dZ = pm.JustText("<dX>"),\
           pm.JustText("<dY>"),\
           pm.JustText("<dZ>")

CellRatio = { 'X' : 2.0, 'Y' : 3.0, 'Z' : 0.75 }
#CellRatio = { 'X' : 1.00, 'Y' : 0.50, 'Z' : 0.75 }

dX.value = mo.buildDXYZ(LX.value,CellRatio['X'],nX.value,hasBump=True)
dY.value = mo.buildDXYZ(LY.value,CellRatio['Y'],nY.value,hasBump=True)

if nZ.value == 1:
    dZ.value = LZ.strValue
else:
    dZ.value = mo.buildDXYZ(LZ.value,CellRatio['Z'],nZ.value,hasBump=False)

In [11]:
# Time config
endTime = pm.Real("<endTime>",value=100.,units="d")

In [12]:
## Bioparticle
kAtt,kDet = pm.Real(tag="<katt>",value=1.0E-30,units="1/s",mathRep="$$k_{att}$$"),\
            pm.Real(tag="<kdet>",value=1.0E-30,units="1/s",mathRep="$$k_{det}$$")

decayAq,decayIm = pm.Real(tag="<decayAq>",value=1.0E-30,units="1/s",mathRep="$$\lambda_{aq}$$"),\
                  pm.Real(tag="<decayIm>",value=1.0E-30,units="1/s",mathRep="$$\lambda_{im}$$")

In [13]:
caseDict = {
    "Temp":{
        "Reference" : Ref,
        "Atmosphere": Atm,
        "Injection" : Tin },
   "longDisp":LongDisp,
   "Gradient":{
       "X" :GX,
       "Y" :GY,
       "Z" :GZ },
   "L":{
       "X" :LX,
       "Y" :LY,
       "Z" :LZ },
   "k":{
       "X" :kX,
       "Y" :kY,
       "Z" :kZ },
   "theta":theta,
   "outCoord":{
       "X" : { 1 : outX1,
               2 : outX2},
       "Y" : { 1 : outY1,
               2 : outY2},
       "Z" : { 1 : outZ1,
               2 : outZ2}},
   "inCoord":{
       "X" : { 1 : inX1,
               2 : inX2},
       "Y" : { 1 : inY1,
               2 : inY2},
       "Z" : { 1 : inZ1,
               2 : inZ2}},
    "C0":C0,
    "Q":{"In":Qin,
         "Out":Qout},
    "nGrid":{"X":nX,
             "Y":nY,
             "Z":nZ},
    "dGrid":{"X":dX,
             "Y":dY,
             "Z":dZ},
    "endTime":endTime,
    "BIOPARTICLE":{
        "katt" : kAtt,
        "kdet" : kDet,
        "decayAq" : decayAq,
        "decayIm" : decayIm}
           }

In [14]:
import pickle

with open('caseDict.pkl', 'wb') as f:
    pickle.dump(caseDict,f)